<a href="https://colab.research.google.com/github/angelosou/biblioteca_pandas/blob/main/Extra%C3%A7%C3%A3o_e_Tratamento_de_Dados_em_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/data.csv")

In [ ]:
print(df.dtypes)

In [ ]:
print(df.head())

In [ ]:
print(df.isnull().sum())

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df['UnitPrice'])
plt.show()

plt.boxplot(df['Quantity'])
plt.show()

In [ ]:
nulos_customerid = df['CustomerID'].isna().sum()

print("Número de valores nulos em CustomerID:", nulos_customerid)

In [ ]:
if nulos_customerid > 0:
    df.dropna(subset=['CustomerID'], inplace=True)
    print("Linhas com valores nulos em CustomerID foram removidas.")
else:
    print("Não há valores nulos em CustomerID.")

In [ ]:
df = df[df['UnitPrice'] > 0]

In [ ]:
df = df[df['Quantity'] > 0]

In [ ]:
linhas_antes = df.shape[0]

linhas_depois = df.shape[0]
print("Número de linhas removidas:", linhas_antes - linhas_depois)

In [ ]:
duplicadas = df[df.duplicated()]

print("Número de linhas duplicadas:", duplicadas.shape[0])

In [ ]:
if duplicadas.shape[0] > 0:
    df.drop_duplicates(inplace=True)
    print("Linhas duplicadas removidas.")
else:
    print("Não há linhas duplicadas no DataFrame.")

In [ ]:
print("Tipo de dado atual de CustomerID:", df['CustomerID'].dtype)

df['CustomerID'] = df['CustomerID'].astype(str)

print("Novo tipo de dado de CustomerID:", df['CustomerID'].dtype)

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df['Quantity'])
plt.title("Boxplot da quantidade de itens")
plt.show()

plt.boxplot(df['UnitPrice'])
plt.title("Boxplot do preço unitário")
plt.show()

In [ ]:
df['TotalCompra'] = df['Quantity'] * df['UnitPrice']
print(df.head())
df['TotalCompra'] = df['TotalCompra'].apply(lambda x: f"R${x:.2f}")

In [ ]:
data_ultima_compra = df['InvoiceDate'].max()

print("Data da última compra:", data_ultima_compra)

In [ ]:
df['CustomerID'] = df['CustomerID'].astype(str)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df = df[ (df['UnitPrice'] > 0) & (df['Quantity'] > 0) ]
df.drop_duplicates(inplace=True)
df['TotalCompra'] = df['Quantity'] * df['UnitPrice']
data_ultima_compra = df['InvoiceDate'].max()
top_10_paises = df.groupby('Country')['TotalCompra'].sum().sort_values(ascending=False).head(10)
top_10_paises.plot(kind='bar', figsize=(10,6))

In [ ]:
produtos_vendidos = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False)
top_10_produtos = produtos_vendidos.head(10)
top_10_produtos.plot(kind='bar', figsize=(10,6))
plt.xlabel("Descrição do produto")
plt.ylabel("Quantidade vendida")
plt.title("Top 10 produtos mais vendidos")
plt.show()

In [ ]:
df['Mes'] = df['InvoiceDate'].dt.month
vendas_por_mes = df.groupby('Mes')['TotalCompra'].sum()
vendas_por_mes.plot(kind='line', figsize=(10,6))
plt.xlabel("Mês")
plt.ylabel("Valor total das vendas")
plt.title("Valor de venda total por mês")
plt.show()

In [ ]:
df_top_paises = df[df['Country'].isin(top_10_paises.index)]

vendas_mes_pais = df_top_paises.groupby(['Mes', 'Country'])['TotalCompra'].sum()

vendas_mes_pais.unstack().plot(kind='line', figsize=(10,6))
plt.xlabel("Mês")
plt.ylabel("Valor total das vendas")
plt.title("Valor de venda total por mês e por país (top 10)")
plt.legend(title="País", loc='upper right')
plt.show()

In [ ]:
ultima_compra_cliente = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
ultima_compra_cliente.rename(columns={'InvoiceDate': 'UltimaCompra'}, inplace=True)
print(ultima_compra_cliente.head())

In [ ]:
rfm_table = df.groupby('CustomerID').agg({
    'InvoiceNo': 'count',
    'TotalCompra': 'mean'
}).reset_index()
rfm_table.rename(columns={'InvoiceNo': 'F',
                         'TotalCompra': 'M'}, inplace=True)
print(rfm_table.head())

In [ ]:
rfm_table = pd.merge(rfm_table, ultima_compra_cliente, on='CustomerID')
rfm_table['R'] = (data_ultima_compra - rfm_table['UltimaCompra']).dt.days
print(rfm_table.head())